# Example 2: Metabolite workflow

This notebook provides insights on how to use the tool if you have list of metabolites.

# Import modules

In [1]:
import pandas as pd

from pyBiodatafuse import id_mapper
from pyBiodatafuse.analyzer.summarize import BioGraph
from pyBiodatafuse.annotators import molmedb, opentargets
from pyBiodatafuse.graph import saver
from pyBiodatafuse.utils import combine_sources, create_or_append_to_metadata

# Load list of metabolites/chemicals/compounds

In [2]:
metabolites_of_interest = """100208
10040286
10041551
10025195
5291"""

metabolite_list = metabolites_of_interest.split("\n")
len(metabolite_list)

5

In [3]:
data_input = pd.DataFrame(metabolite_list, columns=["identifier"])
data_input.head()

,identifier
0,100208
1,10040286
2,10041551
3,10025195
4,5291


### Entity resolution using BridgeDB

In [4]:
bridgdb_df, bridgdb_metadata = id_mapper.bridgedb_xref(
    identifiers=data_input,
    input_species="Human",
    input_datasource="PubChem Compound",
    output_datasource="All",
)
bridgdb_df.head()

,identifier,identifier.source,target,target.source
0,100208,PubChem-compound,90560,ChemSpider
1,100208,PubChem-compound,100208,PubChem Compound
2,100208,PubChem-compound,HMDB0244377,HMDB
3,100208,PubChem-compound,OFDNQWIFNXBECV-UHFFFAOYSA-N,InChIKey
4,100208,PubChem-compound,C11280,KEGG Compound


### Transporter inhibited from MolMeDB

In [5]:
(
    molmedb_transporter_inhibited_df,
    molmedb_transporter_inhibited_metadata,
) = molmedb.get_compound_gene_inhibitor(bridgedb_df=bridgdb_df)
molmedb_transporter_inhibited_df.head()

,identifier,identifier.source,target,target.source,MolMeDB_transporter_inhibited
0,100208,PubChem-compound,OFDNQWIFNXBECV-UHFFFAOYSA-N,InChIKey,"[{'uniprot_trembl_id': 'P08183', 'hgnc_symbol'..."
1,10025195,PubChem-compound,LEJRLSZVESQKJK-UHFFFAOYSA-N,InChIKey,"[{'uniprot_trembl_id': nan, 'hgnc_symbol': nan..."
2,10040286,PubChem-compound,FYGREZKTJIXWIH-UHFFFAOYSA-N,InChIKey,"[{'uniprot_trembl_id': nan, 'hgnc_symbol': nan..."
3,10041551,PubChem-compound,OVVBIIBBRZVPAL-UHFFFAOYSA-N,InChIKey,"[{'uniprot_trembl_id': 'P23975', 'hgnc_symbol'..."
4,5291,PubChem-compound,KTUFNOKKBVMGRW-UHFFFAOYSA-N,InChIKey,"[{'uniprot_trembl_id': 'O15244', 'hgnc_symbol'..."


### Disease from OpenTargets

In [6]:
(
    opentargets_df,
    opentargets_metadata,
) = opentargets.get_compound_disease_interactions(bridgedb_df=bridgdb_df)
opentargets_df.head()

/var/folders/x2/1tdvwk7x2_n98kzwy7rnt3t00000gn/T/ipykernel_25626/3172929253.py:4: UserWarning: The intermediate_df in OpenTargets_diseases annotatur should be checked, please create an issue on https://github.com/BioDataFuse/pyBiodatafuse/issues/.
  ) = opentargets.get_compound_disease_interactions(bridgedb_df=bridgdb_df)


,identifier,identifier.source,target,target.source,OpenTargets_diseases
0,100208,PubChem-compound,100208,PubChem Compound,"[{'disease_name': nan, 'therapeutic_areas': na..."
1,10025195,PubChem-compound,10025195,PubChem Compound,"[{'disease_name': nan, 'therapeutic_areas': na..."
2,10040286,PubChem-compound,10040286,PubChem Compound,"[{'disease_name': nan, 'therapeutic_areas': na..."
3,10041551,PubChem-compound,10041551,PubChem Compound,"[{'disease_name': nan, 'therapeutic_areas': na..."
4,5291,PubChem-compound,5291,PubChem Compound,"[{'disease_name': 'colonic neoplasm', 'therape..."


# Create the BDF graph

In [7]:
combined_df = combine_sources(
    bridgdb_df, df_list=[opentargets_df, molmedb_transporter_inhibited_df]
)
combined_df.head()

,identifier,identifier.source,target,target.source,OpenTargets_diseases,MolMeDB_transporter_inhibited
0,100208,PubChem-compound,100208,PubChem Compound,"[{'disease_name': nan, 'therapeutic_areas': na...","[{'uniprot_trembl_id': 'P08183', 'hgnc_symbol'..."
1,10040286,PubChem-compound,10040286,PubChem Compound,"[{'disease_name': nan, 'therapeutic_areas': na...","[{'uniprot_trembl_id': nan, 'hgnc_symbol': nan..."
2,10041551,PubChem-compound,10041551,PubChem Compound,"[{'disease_name': nan, 'therapeutic_areas': na...","[{'uniprot_trembl_id': 'P23975', 'hgnc_symbol'..."
3,10025195,PubChem-compound,10025195,PubChem Compound,"[{'disease_name': nan, 'therapeutic_areas': na...","[{'uniprot_trembl_id': nan, 'hgnc_symbol': nan..."
4,5291,PubChem-compound,5291,PubChem Compound,"[{'disease_name': 'colonic neoplasm', 'therape...","[{'uniprot_trembl_id': 'O15244', 'hgnc_symbol'..."


In [8]:
combined_metadata = create_or_append_to_metadata(
    bridgdb_metadata, [opentargets_metadata, molmedb_transporter_inhibited_metadata]
)

# Saving the graph

In [9]:
import os

DATA_DIR = "./data/metabolite_workflow"
os.makedirs(DATA_DIR, exist_ok=True)

In [10]:
pygraph = saver.save_graph(
    combined_df=combined_df,
    combined_metadata=combined_metadata,
    graph_name="metabolite_workflow",
    graph_dir=DATA_DIR,
)

Combined DataFrame saved in ./data/metabolite_workflow/metabolite_workflow_df.pkl
Metadata saved in ./data/metabolite_workflow/metabolite_workflow_metadata.pkl
Building graph: 0it [00:00, ?it/s]
Graph is built successfully
Graph saved in ./data/metabolite_workflow/metabolite_workflow_graph.pkl and ./data/metabolite_workflow/metabolite_workflow_graph.gml
Graph saved in ./data/metabolite_workflow/metabolite_workflow_graph.edgelist


# Graph statistics

In [ ]:
graph_obj = BioGraph(graph=pygraph)
graph_obj.graph_summary

In [ ]:
graph_obj.count_nodes_by_data_source(plot=True)

In [ ]:
graph_obj.count_edge_by_data_source(plot=True)